In [3]:
import sys

sys.path.append("..")
import warnings
from dataclasses import dataclass
from pathlib import Path
from tempfile import mkdtemp
from typing import List, Tuple

import numpy as np
import pandas as pd
import SimpleITK as sitk
from deepdrr import MobileCArm, Volume
from deepdrr.projector import Projector
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from utils.cv import crop, min_max_normalize
from ct.utils import read_dcm
from utils.utils import save_img, track
import deepdrr
import pyvista
from utils.plot import plot_grid
from utils.cv import remove_border, crop
from itertools import product

import numpy as np
import SimpleITK as sitk
from deepdrr import MobileCArm, Volume
from deepdrr.geo import FrameTransform
from deepdrr.projector import Projector
import pickle

pyvista.start_xvfb()


In [2]:
def get_drr(img: sitk.Image, move_by: int = 750) -> np.ndarray:
    # Extract pixel and material data from sitk
    hu_values = np.transpose(sitk.GetArrayFromImage(img), [2, 1, 0]).astype(float)
    data = Volume._convert_hounsfield_to_density(hu_values)
    materials = Volume.segment_materials(
        hu_values, use_thresholding=True, use_cached=False
    )

    # Set up nifti affine
    affine = np.zeros((4, 4))
    origin = img.GetOrigin()
    direction = np.array(img.GetDirection()).reshape((3, 3))
    for (i, j) in product(range(3), repeat=2):
        if direction[i][j] != 0:
            affine[i][j] = direction[i][j]
    affine[:3, 3] = origin
    affine[:2, :] *= np.sign(origin[2]) + int(origin[2] == 0)
    affine[3, 3] = 1.0
    anatomical_from_ijk = FrameTransform(affine)
    volume = Volume(data, materials, anatomical_from_ijk, None, "RAS")
    volume.facedown()
    carm = MobileCArm()
    carm.reposition(volume.center_in_world)
    carm.move_to(alpha=0, beta=-180, degrees=True)
    carm.move_by((0, 0, move_by))
    warnings.filterwarnings("ignore")
    return deepdrr.vis.show(volume, carm, full=[True, True]);

In [12]:
img1, img2 = read_dcm(pickle.load(open("../output/covid_ct_train_meta.pkl", "rb"))["lung"].tolist()[0])

In [13]:
get_drr(img1)

NameError: name 'vol' is not defined